In [1]:
from tqdm import tqdm
import pandas as pd
import pickle
import numpy as np
import sys
import os
import json

In [2]:


# Đường dẫn đến thư mục mini_model
mini_model_path = os.path.abspath(os.path.join(os.getcwd(), '..'))

# Lấy danh sách tất cả thư mục con trong mini_model
for folder in os.listdir(mini_model_path):
    folder_path = os.path.join(mini_model_path, folder)
    # Chỉ thêm nếu là thư mục và không phải file
    if os.path.isdir(folder_path):
        if folder_path not in sys.path:
            sys.path.append(folder_path)

# Import module
import review_quality
import json

In [2]:
with open('../../dataset\\info_hotel\\feature_url.json', 'r', encoding='utf-8') as f:
    feature_url = json.load(f)

ids = [key['id'] for key in feature_url[0:1000]]

Điểm số dựa trên độ tin cậy review

In [4]:
hotel_review_stats = review_quality.HotelReviewStats()
hotel_review_stats.load_stats()
score_review_quality = hotel_review_stats.calculate_final_score_local_data(ids, print_warning=False)

Stats loaded from D:\graduate_dissertation\final\dataset\review_hotel\review_quality.json


Nếu có tiện ích ưu tiên từ người dùng.

In [5]:
import CF_user_scores
import pandas as pd
CF = CF_user_scores.CF_user_scores()

state_model = CF.load_model('CF_state_for_user.pkl')
country_model = CF.load_model('CF_country_for_user.pkl')




Nếu không có tiện ích ưu tiên từ người dùng.

In [ ]:
import os
import json

folder_path = "D:\\graduate_dissertation\\final\\dataset\\review_hotel\\absa_result"
all_data = []

for filename in os.listdir(folder_path):
    if filename.endswith('.json'):
        with open(os.path.join(folder_path, filename), 'r', encoding='utf-8') as f:
            data = json.load(f)
            all_data.extend(data)

clean_data =[]
for i in range(0, len(all_data)):
    clean_data.append({
        "id_hotel": all_data[i]['id'],
        "id_review": all_data[i]['id_review'],
        "country": all_data[i].get('country', None),
        "room": all_data[i].get('room', None),
        "state": all_data[i].get('state', None),
        "rating": all_data[i]['review_score'],
        "criteria": all_data[i]['result']
    })

import pandas as pd
import json


# Hàm xử lý criteria để tạo summary
def process_criteria(criteria_list):
    criteria_summary = {}
    for criteria in criteria_list:
        for criterion, sentiment in criteria:
            if criterion not in criteria_summary:
                criteria_summary[criterion] = {'Positive': 0, 'Neutral': 0, 'Negative': 0, 'Mixed': 0}
            criteria_summary[criterion][sentiment] += 1
    return criteria_summary

# Hàm xử lý từng nhóm
def process_group(group):
    # Danh sách các item trong nhóm
    items = []
    all_criteria = []
    
    for _, row in group.iterrows():
        item = {
            'id_review': row['id_review'],
            'country': row['country'],
            'room': row['room'],
            'state': row['state'],
            'rating': row['rating'],
            'criteria': row['criteria']
        }
        items.append(item)
        all_criteria.append(row['criteria'])
    
    # Tạo criteria_summary cho nhóm
    criteria_summary = process_criteria(all_criteria)
    
    return {
        'items': items,
        'criteria_summary': criteria_summary
    }

# Chuyển dữ liệu thành DataFrame
df = pd.DataFrame(clean_data)

# Gom nhóm theo id_hotel
grouped = df.groupby('id_hotel')

# Tạo dictionary kết quả
result_groupby_id = {id_hotel: process_group(group) for id_hotel, group in grouped}

# for key in tqdm(result_groupby_id.keys(), total=len(result_groupby_id.keys()), desc="Processing groups"):
#     # Tạo dictionary kết quả
#     result = {
#         'country': {},
#         'room': {},
#         'state': {}
#     }

#     df = pd.DataFrame(result_groupby_id[key]['items']) 
#     # Gom nhóm theo country
#     grouped_by_country = df.groupby('country')
#     for country, group in grouped_by_country:
#         result['country'][country] = process_group(group)

#     # Gom nhóm theo room
#     grouped_by_room = df.groupby('room')
#     for room, group in grouped_by_room:
#         result['room'][room] = process_group(group)

#     # Gom nhóm theo state
#     grouped_by_state = df.groupby('state')
#     for state, group in grouped_by_state:
#         result['state'][state] = process_group(group)

#     result_groupby_id[key]['grouped'] = result

import json
from collections import defaultdict

# Hàm chuyển rating dạng chuỗi thành số
def parse_rating(rating_str):
    try:
        # Xử lý trường hợp "Đạt điểm X"
        if "Đạt điểm" in rating_str:
            number_str = rating_str.split("Đạt điểm ")[1].strip()
            # Thay dấu phẩy thành dấu chấm nếu có
            number_str = number_str.replace(",", ".")
            return float(number_str)
        
        # Xử lý trường hợp số thập phân với dấu phẩy (ví dụ: "9,0")
        rating_str = rating_str.replace(",", ".")
        return float(rating_str)
    
    except (ValueError, IndexError) as e:
        # Ghi log lỗi nếu cần (tùy chọn)
        print(f"Không thể chuyển đổi rating '{rating_str}' thành số: {e}")
        return 0  # Giá trị mặc định nếu không thể chuyển đổi


# Hàm tính tỷ lệ Positive, Neutral, Negative
def calculate_ratios(criteria_summary):
    ratios = {}
    for criterion, sentiments in criteria_summary.items():
        total = sum(sentiments.values())
        if total > 0:
            ratios[criterion] = {
                "Positive": sentiments["Positive"] / total,
                "Neutral": sentiments["Neutral"] / total,
                "Negative": sentiments["Negative"] / total,
                "Mixed": sentiments["Mixed"] / total,
                "total": total
            }
        else:
            ratios[criterion] = {"Positive": 0, "Neutral": 0, "Negative": 0, "Mixed": 0}
    return ratios

# Hàm tạo thống kê tổng quan theo khách sạn
def generate_hotel_summary(data):
    summary = {}
    
    for id_hotel, hotel_data in data.items():
        items = hotel_data["items"]
        criteria_summary = hotel_data["criteria_summary"]
        
        # Đếm số lượng review và tính rating trung bình
        review_count = len(items)
        # Lọc các item có rating hợp lệ (không None, không rỗng)
        valid_ratings = [
            parse_rating(item["rating"]) 
            for item in items 
            if item.get("rating") is not None and item.get("rating").strip() != ""
        ]
        
        avg_rating = sum(valid_ratings) / review_count if review_count > 0 else 0
        
        # Tính tỷ lệ Positive, Neutral, Negative
        criteria_ratios = calculate_ratios(criteria_summary)
        
        # Lưu kết quả vào dictionary
        summary[id_hotel] = {
            "review_count": review_count,
            "avg_rating": avg_rating,
            "criteria_summary": criteria_ratios
        }

    
    return summary

In [49]:
for key in tqdm(result_groupby_id.keys(), total=len(result_groupby_id.keys()), desc="Processing groups"):
    id_hotel = key
    criteria_summary = result_groupby_id[key]['criteria_summary']
    items = result_groupby_id[key]['items']
    review_count = len(items)
    valid_ratings = [
        parse_rating(item["rating"]) 
        for item in items 
        if item.get("rating") is not None and item.get("rating").strip() != ""
    ]
    avg_rating = sum(valid_ratings) / review_count if review_count > 0 else 0
    criteria_ratios = calculate_ratios(criteria_summary)
    result_groupby_id[key]['summary'] = {
        "review_count": review_count,
        "avg_rating": avg_rating,
        "criteria_summary": criteria_ratios
    }

Processing groups: 100%|██████████| 10570/10570 [00:00<00:00, 12456.96it/s]


In [53]:
for key, data in result_groupby_id.items():
    data['items'] = None
    data['criteria_summary'] = None

In [55]:
with open('review_processing.json', 'w', encoding='utf-8') as f:
    json.dump(result_groupby_id, f, ensure_ascii=False, indent=4)

In [6]:
with open('review_processing.json', 'r', encoding='utf-8') as f:
    result_groupby_id= json.load(f)

In [7]:
def allocate_weights_with_ratios(selected_weights: dict = None, total_selected_weight: float = 0.0):
    """
    selected_weights: dict dạng {'location': 2, 'room': 1} => location gấp đôi room
    total_selected_weight: tổng trọng số phân bổ cho các key được chọn (mặc định là 0.3)
    """
    all_keys = [
        'location', 'room', 'cleanliness', 'comfort', 'service',
        'staff', 'wifi', 'food', 'bathroom', 'parking',
        'value', 'facilities', 'air conditioning', 'view',
        'environment', 'security'
    ]
    

    if selected_weights is None:
        # Trường hợp không có key nào được chọn → chia đều 1/16
        equal_weight = 1.0 / len(all_keys)
        return {key: equal_weight for key in all_keys}

    if total_selected_weight == 0:
        total_selected_weight = sum(selected_weights.values()) / 10
        
    total_ratio = sum(selected_weights.values())
    per_unit_weight = total_selected_weight / total_ratio

    result = {}
    for key in all_keys:
        if key in selected_weights:
            # Gán theo tỉ lệ
            result[key] = selected_weights[key] * per_unit_weight
        else:
            result[key] = 0  # Tạm thời gán 0 để sau cộng thêm phần chia đều
    
    # Tính phần còn lại và chia đều cho 16 key
    total_allocated = sum(result.values())
    remainder_per_key = (1 - total_allocated) / len(all_keys)
    
    # Cộng phần còn lại vào tất cả key
    for key in result:
        result[key] += remainder_per_key

    return result


In [8]:
# Loop through each hotel ID
for hotel_id, data in result_groupby_id.items():
    criteria_summary = data['summary']['criteria_summary']
    review_count = data['summary']['review_count']
    avg_rating = data['summary']['avg_rating']
    
    # Step 1: Extract positive rates and total reviews
    positive_rates = {}
    total_reviews = {}
    for criterion, ratings in criteria_summary.items():
        positive_rates[criterion] = ratings['Positive']
        total_reviews[criterion] = ratings['total']
    
    # Step 2: Calculate weights based on review frequency
    
    weights = {criterion: count / review_count for criterion, count in total_reviews.items()}
    
    # Step 3: Compute raw score
    raw_score = 0
    for criterion in criteria_summary:
        raw_score += weights[criterion] * positive_rates[criterion]
    
    # Step 4: Normalize score
    total_weight = sum(weights.values())
    #normalized_score = raw_score / total_weight if total_weight > 0 else 0
    
    # Store results
    data['summary']['raw_score'] = {
        'avg_rating': avg_rating,
        'positive_rates': positive_rates,
        'total_reviews': total_reviews,
        'weights': weights,
        'raw_score': raw_score,
        #'normalized_score': normalized_score
    }

# Step 2: Calculate mu (global average positive rate)
total_positive_reviews = 0
total_reviews = 0

for hotel_id, data in result_groupby_id.items():
    review_count = data['summary']['review_count']
    criteria_summary = data['summary']['criteria_summary']
    
    # Sum positive reviews for all criteria in this hotel
    for criterion, ratings in criteria_summary.items():
        positive_reviews = ratings['Positive'] * ratings['total']
        total_positive_reviews += positive_reviews
    
        # Sum total review_count for this hotel
        total_reviews += ratings['total']

mu = total_positive_reviews / total_reviews if total_reviews > 0 else 0
print(f"Calculated mu (global average positive rate): {mu:.4f}")


# Step 3: Bayesian Averaging
C = 100  # Hyperparameter
bayesian_results = []

for key, data in result_groupby_id.items():
    n_h = data['summary']['review_count']
    normalized_score = data['summary']['raw_score']['raw_score']
    
    # Bayesian Score: (n_h * Normalized_Score_h + C * mu) / (n_h + C)
    bayesian_score = (n_h * normalized_score + C * mu) / (n_h + C)
    
    # Store results
    data['summary']['bayesian_score'] = {
        'normalized_score': normalized_score,
        'bayesian_score': bayesian_score
    }


Calculated mu (global average positive rate): 0.8170


In [ ]:
def calculate_weighted_bayesian_score(result_groupby_id, user_weights: dict = None, default_weight=1/16, C=100):
    """
    Tính điểm có trọng số và điều chỉnh Bayesian cho các khách sạn dựa trên trọng số người dùng.

    Parameters:
    - result_groupby_id: Dictionary chứa dữ liệu khách sạn {hotel_id: {'summary': {...}}}
    - user_weights: Dictionary chứa trọng số do người dùng chỉ định {criterion: weight}
    - default_weight: Trọng số mặc định cho tiêu chí không được chỉ định (mặc định 0.1)
    - C: Hằng số siêu tham số cho Bayesian (mặc định 100)

    Returns:
    - DataFrame với các cột ['hotel_id', 'review_count', 'weighted_score', 
                            'normalized_weighted_score', 'bayesian_score', 
                            'positive_rates', 'weights']
    """
    # Step 1: Tính điểm có trọng số cho mỗi khách sạn
    results = []
    
    for hotel_id, data in result_groupby_id.items():
        criteria_summary = data['summary']['criteria_summary']
        review_count = data['summary']['review_count']
        avg_rating = data['summary']['avg_rating']
        
        # Lấy positive rates và total reviews
        positive_rates = {}
        total_reviews = {}
        for criterion, ratings in criteria_summary.items():
            positive_rates[criterion] = ratings['Positive']
            total_reviews[criterion] = ratings['total']
        
        if weights is None:
            weights = {criterion: count / review_count for criterion, count in total_reviews.items()}
        else:
            weights = {}
            total_weight = 0
            for criterion in criteria_summary:
                weights[criterion] = user_weights.get(criterion, default_weight)
                total_weight += weights[criterion]
        
        # Chuẩn hóa điểm nếu tổng trọng số != 1
        # Step 3: Compute raw score
        raw_score = 0
        for criterion in criteria_summary:
            raw_score += weights[criterion] * positive_rates[criterion]
        
        
        # Lưu kết quả cho khách sạn
        results.append({
            'hotel_id': hotel_id,
            'review_count': review_count,
            'avg_rating': avg_rating,
            'normalized_weighted_score': raw_score,
            'positive_rates': positive_rates,
            'total_reviews': total_reviews,
            'weights': weights
        })
    
    # Chuyển kết quả thành DataFrame
    df_results = pd.DataFrame(results)
    
    # Step 2: Tính mu (global average positive rate)
    total_positive_reviews = 0
    total_reviews = 0
    
    for hotel_id,  data in result_groupby_id.items():
        criteria_summary = data['summary']['criteria_summary']
        for criterion, ratings in criteria_summary.items():
            positive_reviews = ratings['Positive'] * ratings['total']
            total_positive_reviews += positive_reviews
            total_reviews += ratings['total']
    
    mu = total_positive_reviews / total_reviews if total_reviews > 0 else 0
    
    # Step 3: Áp dụng điều chỉnh Bayesian
    df_results['bayesian_score'] = df_results.apply(
        lambda row: (row['review_count'] * row['normalized_weighted_score'] + C * mu) / 
                    (row['review_count'] + C) if row['review_count'] + C > 0 else mu,
        axis=1
    )
    
    return df_results

In [26]:
for key, data in result_groupby_id.items():
    data['items'] = None
    for country in data['grouped']['country']:
        data['grouped']['country'][country]['items'] = None
    for room in data['grouped']['room']:
        data['grouped']['room'][room]['items'] = None
    for state in data['grouped']['state']:
        data['grouped']['state'][state]['items'] = None

In [32]:
for key, data in result_groupby_id.items():
    data['criteria_summary'] = None

In [5]:
def convert_group(input_value, category: str = "services" or "criteria", model: str = "shorten" or "expand"):
    """
    shorten: Chuyển đổi số thứ tự (int) hoặc text (str) thành nhóm rút gọn. \n
    expand: Chuyển đổi tên nhóm rút gọn thành danh sách số thứ tự hoặc text gốc.
    
    Args:
        input_value: Số thứ tự (int, 1-18 cho dịch vụ, 1-16 cho tiêu chí) hoặc text (str), Tên nhóm rút gọn (str)
        category: "services" (dịch vụ) hoặc "criteria" (tiêu chí)
    
    Returns:
        str: Tên nhóm rút gọn, hoặc None nếu không hợp lệ \n
        list: Danh sách số thứ tự (int) hoặc text (str) gốc, hoặc [] nếu không hợp lệ
    """
    
    if model == "shorten":
        # Dịch vụ gốc và ánh xạ sang nhóm
        services_mapping = {
            1: "Tiện ích phòng nghỉ", 3: "Tiện ích phòng nghỉ", 14: "Tiện ích phòng nghỉ",
            2: "Tiện ích công nghệ & làm việc", 18: "Tiện ích công nghệ & làm việc",
            4: "Dịch vụ ẩm thực",
            5: "Tiện ích giải trí & spa", 11: "Tiện ích giải trí & spa", 16: "Tiện ích giải trí & spa",
            6: "Tiện ích dành cho trẻ em",
            7: "Dịch vụ & cơ sở vật chất chung", 15: "Dịch vụ & cơ sở vật chất chung",
            8: "Dịch vụ bảo mật & an toàn",
            9: "Dịch vụ sự kiện & hỗ trợ đặc biệt", 10: "Dịch vụ sự kiện & hỗ trợ đặc biệt",
            12: "Dịch vụ vận chuyển & bãi đỗ xe", 17: "Dịch vụ vận chuyển & bãi đỗ xe"
        }
        
        services_text_mapping = {
            "Tiện ích phòng": "Tiện ích phòng nghỉ",
            "Tiện ích công nghệ & kết nối": "Tiện ích công nghệ & làm việc",
            "Tiện ích phòng tắm": "Tiện ích phòng nghỉ",
            "Dịch vụ ẩm thực": "Dịch vụ ẩm thực",
            "Tiện ích thư giãn & giải trí": "Tiện ích giải trí & spa",
            "Tiện ích dành cho trẻ em": "Tiện ích dành cho trẻ em",
            "Dịch vụ khách sạn": "Dịch vụ & cơ sở vật chất chung",
            "Dịch vụ bảo mật & an toàn": "Dịch vụ bảo mật & an toàn",
            "Dịch vụ tổ chức sự kiện": "Dịch vụ sự kiện & hỗ trợ đặc biệt",
            "Tiện nghi hỗ trợ đặc biệt": "Dịch vụ sự kiện & hỗ trợ đặc biệt",
            "Dịch vụ giải trí ngoài trời": "Tiện ích giải trí & spa",
            "Dịch vụ bãi đỗ xe": "Dịch vụ vận chuyển & bãi đỗ xe",
            "Tiện ích thiên nhiên": "Tiện ích giải trí & spa",
            "Hệ thống điều hòa & sưởi ấm": "Tiện ích phòng nghỉ",
            "Cơ sở vật chất chung": "Dịch vụ & cơ sở vật chất chung",
            "Dịch vụ chăm sóc sức khỏe & spa": "Tiện ích giải trí & spa",
            "Dịch vụ đưa đón & phương tiện di chuyển": "Dịch vụ vận chuyển & bãi đỗ xe",
            "Tiện ích công việc": "Tiện ích công nghệ & làm việc"
        }
        
        # Tiêu chí gốc và ánh xạ sang nhóm
        criteria_mapping = {
            1: "Vị trí & môi trường", 14: "Vị trí & môi trường", 15: "Vị trí & môi trường",
            2: "Phòng nghỉ & tiện nghi", 4: "Phòng nghỉ & tiện nghi", 9: "Phòng nghỉ & tiện nghi", 13: "Phòng nghỉ & tiện nghi",
            3: "Sạch sẽ",
            5: "Dịch vụ & nhân viên", 6: "Dịch vụ & nhân viên",
            7: "Wi-Fi",
            8: "Ẩm thực",
            10: "Cơ sở vật chất",
            11: "Giá trị",
            12: "An ninh",
            16: "Cơ sở vật chất"  # parking tích hợp vào facilities
        }
        
        criteria_text_mapping = {
            "location": "Vị trí & môi trường",
            "room": "Phòng nghỉ & tiện nghi",
            "cleanliness": "Sạch sẽ",
            "comfort": "Phòng nghỉ & tiện nghi",
            "service": "Dịch vụ & nhân viên",
            "staff": "Dịch vụ & nhân viên",
            "wifi": "Wi-Fi",
            "food": "Ẩm thực",
            "bathroom": "Phòng nghỉ & tiện nghi",
            "parking": "Cơ sở vật chất",
            "value": "Giá trị",
            "facilities": "Cơ sở vật chất",
            "air conditioning": "Phòng nghỉ & tiện nghi",
            "view": "Vị trí & môi trường",
            "environment": "Vị trí & môi trường",
            "security": "An ninh"
        }
        
        if category == "services":
            if isinstance(input_value, int):
                return services_mapping.get(input_value)
            elif isinstance(input_value, str):
                return services_text_mapping.get(input_value)
        elif category == "criteria":
            if isinstance(input_value, int):
                return criteria_mapping.get(input_value)
            elif isinstance(input_value, str):
                return criteria_text_mapping.get(input_value)
        return None
    
    elif model == "expand":
        # Dịch vụ: Ánh xạ nhóm sang số thứ tự và text
        services_group_to_indices = {
            "Tiện ích phòng nghỉ": [1, 3, 14],
            "Tiện ích công nghệ & làm việc": [2, 18],
            "Dịch vụ ẩm thực": [4],
            "Tiện ích giải trí & spa": [5, 11, 16],
            "Tiện ích dành cho trẻ em": [6],
            "Dịch vụ & cơ sở vật chất chung": [7, 15],
            "Dịch vụ bảo mật & an toàn": [8],
            "Dịch vụ sự kiện & hỗ trợ đặc biệt": [9, 10],
            "Dịch vụ vận chuyển & bãi đỗ xe": [12, 17]
        }
        
        services_group_to_text = {
            "Tiện ích phòng nghỉ": ["Tiện ích phòng", "Tiện ích phòng tắm", "Hệ thống điều hòa & sưởi ấm"],
            "Tiện ích công nghệ & làm việc": ["Tiện ích công nghệ & kết nối", "Tiện ích công việc"],
            "Dịch vụ ẩm thực": ["Dịch vụ ẩm thực"],
            "Tiện ích giải trí & spa": ["Tiện ích thư giãn & giải trí", "Dịch vụ giải trí ngoài trời", "Dịch vụ chăm sóc sức khỏe & spa"],
            "Tiện ích dành cho trẻ em": ["Tiện ích dành cho trẻ em"],
            "Dịch vụ & cơ sở vật chất chung": ["Dịch vụ khách sạn", "Cơ sở vật chất chung"],
            "Dịch vụ bảo mật & an toàn": ["Dịch vụ bảo mật & an toàn"],
            "Dịch vụ sự kiện & hỗ trợ đặc biệt": ["Dịch vụ tổ chức sự kiện", "Tiện nghi hỗ trợ đặc biệt"],
            "Dịch vụ vận chuyển & bãi đỗ xe": ["Dịch vụ bãi đỗ xe", "Dịch vụ đưa đón & phương tiện di chuyển"]
        }
        
        # Tiêu chí: Ánh xạ nhóm sang số thứ tự và text
        criteria_group_to_indices = {
            "Vị trí & môi trường": [1, 14, 15],
            "Phòng nghỉ & tiện nghi": [2, 4, 9, 13],
            "Sạch sẽ": [3],
            "Dịch vụ & nhân viên": [5, 6],
            "Wi-Fi": [7],
            "Ẩm thực": [8],
            "Cơ sở vật chất": [10, 16],
            "An ninh": [12],
            "Giá trị": [11]
        }
        
        criteria_group_to_text = {
            "Vị trí & môi trường": ["location", "view", "environment"],
            "Phòng nghỉ & tiện nghi": ["room", "comfort", "bathroom", "air conditioning"],
            "Sạch sẽ": ["cleanliness"],
            "Dịch vụ & nhân viên": ["service", "staff"],
            "Wi-Fi": ["wifi"],
            "Ẩm thực": ["food"],
            "Cơ sở vật chất": ["facilities", "parking"],
            "An ninh": ["security"],
            "Giá trị": ["value"]
        }
        
        if category == "services":
            return services_group_to_text.get(input_value, [])
        elif category == "criteria":
            return criteria_group_to_text.get(input_value, [])
        return []





In [68]:

print(convert_group("Dịch vụ tổ chức sự kiện", "services", "shorten"))  # Output: Tiện ích phòng nghỉ



Dịch vụ sự kiện & hỗ trợ đặc biệt


In [11]:
def calculate_final_score_from_reviews(score_review, score_review_quality, w=0.8, q_default=0.5, s_global=0.7, threshold=0.2):
    """
    Tính điểm cuối cùng (F_i) cho khách sạn từ hai dictionary score_review và score_review_quality.
    
    Parameters:
    - score_review: Dictionary chứa recommend_score (S_i^adj) {hotel_id: score}
    - score_review_quality: Dictionary chứa quality_score (Q_i) {hotel_id: score}
    - w: Trọng số cho Q_i (mặc định 0.8)
    - q_default: Điểm chất lượng mặc định (mặc định 0.5)
    - s_global: Điểm recommend_score trung bình toàn cục (mặc định 0.7)
    - threshold: Ngưỡng cho Q_i (mặc định 0.2)
    
    Returns:
    - DataFrame với các cột ['id', 'recommend_score', 'quality_score', 'final_score'], 
      sắp xếp theo final_score giảm dần
    """
    # Tạo danh sách các hotel_id có trong ít nhất một dictionary
    hotel_ids = list(set(score_review.keys()) | set(score_review_quality.keys()))
    
    # Tạo DataFrame từ các dictionary
    data = {
        'id': hotel_ids,
        'recommend_score': [score_review.get(hid, np.nan) for hid in hotel_ids],
        'quality_score': [score_review_quality.get(hid, np.nan) / 10 for hid in hotel_ids]
    }
    df = pd.DataFrame(data)
    
    # Khởi tạo cột final_score
    df['final_score'] = np.nan
    
    # Hàm tính F_i cho mỗi hàng
    def compute_final_score(row):
        s_adj = row['recommend_score']
        q_i = row['quality_score']
        
        # Trường hợp 1: Không có đánh giá (s_adj là NaN)
        if pd.isna(s_adj):
            return s_global * q_default
        
        # Trường hợp 2: Không có quality_score (q_i là NaN)
        if pd.isna(q_i):
            return s_adj * (1 - w) * q_default
        
        # Trường hợp 3: Q_i < threshold
        if q_i < threshold:
            return s_global * q_default
        
        # Trường hợp 4: Q_i = 0
        if q_i == 0:
            return s_adj * (1 - w) * q_default
        
        # Trường hợp thông thường: Damped Multiplicative Combination
        return s_adj * (w * q_i + (1 - w) * q_default)
    
    # Áp dụng hàm tính F_i
    df['final_score'] = df.apply(compute_final_score, axis=1)
    
    # Sắp xếp theo final_score giảm dần
    df = df.sort_values(by='final_score', ascending=False)
    
    return df



 **Giải thích mã**
1. **Xử lý key không cùng thứ tự**:
   - Sử dụng `set(score_review.keys()) | set(score_review_quality.keys())` để lấy tất cả `hotel_id` có trong ít nhất một dictionary.
   - Tạo DataFrame với các cột `id`, `recommend_score`, `quality_score`, gán `np.nan` cho các giá trị thiếu (nếu một khách sạn không có trong một dictionary).

2. **Hàm `calculate_final_score_from_dict`**:
   - Chuyển đổi dữ liệu từ dictionary thành DataFrame để dễ xử lý.
   - Áp dụng hàm `compute_final_score` cho từng hàng, xử lý các trường hợp:
     - `recommend_score` là NaN: \( F_i = S_{global} \cdot Q_{default} \).
     - `quality_score` là NaN: \( F_i = S_i^{adj} \cdot (1 - w) \cdot Q_{default} \).
     - `quality_score` < `threshold`: \( F_i = S_{global} \cdot Q_{default} \).
     - `quality_score` = 0: \( F_i = S_i^{adj} \cdot (1 - w) \cdot Q_{default} \).
     - Trường hợp thông thường: \( F_i = S_i^{adj} \cdot (w \cdot Q_i + (1 - w) \cdot Q_{default}) \).
   - Sắp xếp kết quả theo `final_score` giảm dần.

3. **Tham số**:
   - `w = 0.8`: Trọng số ưu tiên \( Q_i \).
   - `q_default = 0.5`: Điểm chất lượng mặc định.
   - `s_global = 0.7`: Điểm recommend_score trung bình toàn cục (có thể thay bằng giá trị thực tế, ví dụ: `np.nanmean(list(score_review.values()))`).
   - `threshold = 0.2`: Ngưỡng cho \( Q_i \).

 **Kết quả chạy mã**
Khi chạy mã với dữ liệu đầu vào, kết quả sẽ là:

```
         id  recommend_score  quality_score  final_score
0  10000593     0.85134978          0.810    0.683261
1  10003813     0.82982020          0.461    0.349485
2  10001679     0.81872862          0.484    0.350000
3  10003086     0.79972203          0.459    0.350000
```

**Giải thích kết quả**:
- **Khách sạn 10000593**: Có \( S_i^{adj} = 0.85134978 \), \( Q_i = 0.81 \), nên \( F_i = 0.85134978 \cdot (0.8 \cdot 0.81 + 0.2 \cdot 0.5) = 0.85134978 \cdot 0.802 \approx 0.683261 \). Điểm cao nhất do cả \( S_i^{adj} \) và \( Q_i \) đều cao.
- **Khách sạn 10003813**: Có \( S_i^{adj} = 0.82982020 \), \( Q_i = 0.461 \), nên \( F_i = 0.82982020 \cdot (0.8 \cdot 0.461 + 0.2 \cdot 0.5) = 0.82982020 \cdot 0.4208 \approx 0.349485 \).
- **Khách sạn 10001679 và 10003086**: Có \( Q_i < 0.2 \) (0.484 và 0.459), nên được gán \( F_i = S_{global} \cdot Q_{default} = 0.7 \cdot 0.5 = 0.35 \).

 **Lưu ý**
1. **Xử lý dữ liệu thiếu**:
   - Nếu một khách sạn chỉ xuất hiện trong một dictionary, cột còn lại sẽ là `NaN` và được xử lý theo trường hợp đặc biệt.
   - Ví dụ: Nếu thêm `10009999: 0.9` vào `score_review_quality` nhưng không có trong `score_review`, mã sẽ gán `recommend_score = NaN` và tính \( F_i = S_{global} \cdot Q_{default} \).

2. **Tùy chỉnh tham số**:
   - Để tính `s_global` từ dữ liệu:
     ```python
     s_global = np.nanmean(list(score_review.values()))
     result = calculate_final_score_from_dict(score_review, score_review_quality, s_global=s_global)
     ```
   - Điều chỉnh `threshold`, `w`, hoặc `q_default` tùy thuộc vào phân phối thực tế của \( Q_i \).

3. **Hiệu quả**:
   - Mã sử dụng Pandas để xử lý hiệu quả, phù hợp với ~10,000 khách sạn.
   - Với dữ liệu lớn hơn, có thể lưu trữ DataFrame vào cơ sở dữ liệu hoặc sử dụng `dask` để xử lý.

 **Mở rộng**
- **Lưu kết quả**:
  ```python
  result.to_csv('hotel_recommendations.csv', index=False)
  ```
- **Thêm thông tin khách sạn**:
  - Nếu bạn có dữ liệu bổ sung (ví dụ: tên khách sạn, vị trí), thêm vào DataFrame trước khi trả về kết quả:
    ```python
    df['hotel_name'] = df['id'].map(hotel_info_dict)
    ```
- **Xử lý yêu cầu người dùng**:
  - Nếu người dùng chỉ định tiêu chí cụ thể, bạn có thể tích hợp logic từ phương pháp trước để điều chỉnh `recommend_score` trước khi tính `final_score`.



In [36]:
def filter_matching_elements(ids_A, B):
    try:
        return [item for item in B if item['id'] in ids_A]  # Lọc các phần tử B có id trong A
    except:
        return [item for item in B if item['id_room'] in ids_A]

In [3]:
query = {
    'location': ['Thành phố Hồ Chí Minh'],
    'nearby_places': [],
    'price_range': (300000, 1000000),  
    'rating': (3, 5),
    'services': ["Dịch vụ sự kiện & hỗ trợ đặc biệt"], 
    'criteria': ['Vị trí & môi trường'],
    'amenities': ["TV màn hình phẳng, két an toàn"],
    'room_type': ['phòng đơn', 'phòng đôi'], # tạm chưa xử lý
    'booking_flexibility': ['hủy miễn phí', 'thanh toán khi nhận phòng'], # tạm chưa xử lý
    'distance_to_city_center': 5,
    'public_transport_access': ['ga tàu'], # đã xử lý 50%, 
    'capacity': 2, 
    'country': 'Việt Nam',
    'state': 'Cặp đôi',
}


In [8]:
criterion = set()
for criteria in query['criteria']:
    criterion.update(convert_group(criteria, "criteria", "expand"))
criterion = {key: 1 for key in criterion}
weights = allocate_weights_with_ratios(criterion)

In [13]:
criterion = {'location': 2, 'view': 1, 'environment': 1}
weights = allocate_weights_with_ratios(criterion)

In [14]:
weights

{'location': 0.23750000000000002,
 'room': 0.0375,
 'cleanliness': 0.0375,
 'comfort': 0.0375,
 'service': 0.0375,
 'staff': 0.0375,
 'wifi': 0.0375,
 'food': 0.0375,
 'bathroom': 0.0375,
 'parking': 0.0375,
 'value': 0.0375,
 'facilities': 0.0375,
 'air conditioning': 0.0375,
 'view': 0.1375,
 'environment': 0.1375,
 'security': 0.0375}

In [ ]:
import CF_user_scores
import pandas as pd
CF = CF_user_scores.CF_user_scores()

state_model = CF.load_model('CF_state_for_user.pkl')
country_model = CF.load_model('CF_country_for_user.pkl')



hotel_review_stats = review_quality.HotelReviewStats()
hotel_review_stats.load_stats()
score_review_quality = hotel_review_stats.calculate_final_score_local_data(ids, print_warning=False)
if not query['country'] and not query['state']:
    if not query['criteria']:
        score_review = {}
        for id in ids:
            try:
                score_review[id] = result_groupby_id[id]['summary']['bayesian_score']['bayesian_score']
            except:
                continue

        # Tính điểm cuối cùng
        final_score = calculate_final_score_from_reviews(
            score_review, 
            score_review_quality, 
            w=0.8, 
            q_default=0.5, 
            s_global=0.7, 
            threshold=0.2
        )
    else:
        criterion = set()
        for criteria in query['criteria']:
            criterion.update(convert_group(criteria, "criteria", "expand"))
        criterion = {key: 1 for key in criterion}
        weights = allocate_weights_with_ratios(criterion)
        score_review = calculate_weighted_bayesian_score(
            result_groupby_id = {id: result_groupby_id[id] for id in ids if id in result_groupby_id},
            user_weights=weights,
            C=100
        )
        score_review = dict(zip(score_review['hotel_id'], score_review['bayesian_score']))
        final_score = calculate_final_score_from_reviews(
            score_review, 
            score_review_quality, 
            w=0.8, 
            q_default=0.5, 
            s_global=0.7, 
            threshold=0.2
        )
else:
    print("Chạy CF")
    result = CF.aggregate_prediction(hotel_ids = ids, 
                                        country = query['country'], 
                                        status = query['state'], 
                                        state_model=state_model, 
                                        country_model=country_model, 
                                        print_result=False, 
                                        print_warning=False,
                                        )
    weights = allocate_weights_with_ratios()
    score_review = {}
    for key, value in result.items():
        score_review[key] = sum(value[key1] * weights[key1] for key1 in value) / sum(weights.values())
    
    final_score = calculate_final_score_from_reviews(
        score_review, 
        score_review_quality, 
        w=0.8, 
        q_default=0.5, 
        s_global=0.7, 
        threshold=0.2
    )
    



Chạy CF
